In [1]:
# coding:u8
import pandas as pd
import numpy as np
import copy,gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score,classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
import seaborn as sns
from sklearn.externals import joblib
import gc

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data = pd.read_pickle('data/step1_5.pkl') #nume

In [3]:
data2 = pd.read_pickle('data/step2.pkl') #ctr
data3 = pd.read_pickle('data/step5.pkl') # deep cosine

In [4]:
data4 = pd.read_pickle('data/step3.pkl') # tfidf 
data5 = pd.read_pickle('data/step7.pkl') # bert distance

In [5]:
data6 = pd.read_pickle('data/step4.pkl') #deepmatch
data7 = pd.read_pickle('data/step6_feature.pkl') # deeptype

In [6]:
data8 = pd.read_pickle('data/step6_v2_feature.pkl') # deeptype

In [7]:
exclude = ['text_id',
 'kb_id',
 'train_mention',
 'label',
 'm_id','type','type_vectors']
for item in data8.columns:
    if item not in exclude:
        data8.rename(columns={item:'v2_'+item},inplace=True)

In [8]:
data8['type_loss_all'] = (data8['v2_type_loss']+data7['type_loss'])/2

In [9]:
data[['text_id','train_mention','kb_id','label','num_candidates']].head(5)

,text_id,train_mention,kb_id,label,num_candidates
0,1,南京南站,130287,0,2
1,1,南京南站,311223,1,2
2,1,高铁,341096,1,1
3,1,南京南站,130287,0,2
4,1,南京南站,311223,1,2


In [10]:
for df in [data2,data3,data4,data5,data6,data7,data8]:
    
    for c in df.columns:
        if c not in data.columns:
            data[c] = df[c]
    del df
    gc.collect()
# for c in data4.columns:
#     if c not in data.columns:
#         data[c] = data4[c]

In [11]:
# # 距离相对特征
# data_rel = []
# kfold = KFold(n_splits=5,shuffle=False,random_state=2019)
# for train_index,test_index in tqdm(kfold.split(data)):
#     print(test_index)
#     stat_train_data,stat_test_data = data.loc[train_index,:],data.loc[test_index,:]
#     dist_columns = ['bert_cos','bert_mse','entity_common', 'numerical_2','numerical_3', 
#                     'numerical_4', 'numerical_5', 'numerical_6', 'numerical_7', 'numerical_8',
#                     'numerical_9', 'numerical_10', 'numerical_11', 'numerical_12', 'numerical_13', 'type_loss']
#     for f in dist_columns:
#         pos = stat_train_data[stat_train_data.label==1]
#         train_group = pos.groupby(['kb_id','train_mention'],as_index=False)[f].agg({'dismean_'+f:'mean',
#                                                                                     'discount'+f:'count'})
#         stat_test_data = stat_test_data.merge(train_group,on=['kb_id','train_mention'],how='left')
    
#     data_rel.append(stat_test_data)
# data_rel = pd.concat(data_rel,axis=0)


In [12]:
# for f in dist_columns:
#     data_rel['rel_'+f] = data_rel['dismean_'+f]-data_rel[f]

In [9]:
# del data
# gc.collect()
# data = data_rel.reset_index(drop=True)

In [17]:
# data.to_pickle('data/data_all.pkl')

In [2]:
# data = pd.read_pickle('data/data_all.pkl')

In [11]:
predictor=data.columns

In [12]:
exclude = ['text_id',
 'kb_id',
 'train_mention',
 'label',
 'm_id','type','type_vectors']
# exclude = []
temp = []
for x in predictor:
    if x not in exclude :temp.append(x)
predictor2 = temp 

In [13]:
param = {
    #'num_leaves':100,
    'objective':'binary',
    'metric':'binary_logloss',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':6,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    #'xgboost_dart_mode':True,
    'colsample_bytree':0.5,
    'n_estimators':5000,
    'min_child_weight':5,
    'subsample_freq':2,
    'num_leaves':32,
    'n_jobs':-1,
}


In [16]:
data.shape

(1566680, 782)

In [14]:
kfold = KFold(n_splits=5,shuffle=False,random_state=2019)
c = 1
first = 0
for train_index,test_index in kfold.split(data):
    if first==0:
        first+=1
        continue
    #train_data,val_data = data.loc[train_index,:],data.loc[test_index,:]
    train_data,val_data = data.loc[train_index,:],data.loc[test_index,:]
    print(train_data.shape,val_data.shape)
    train = lgb.Dataset(train_data[predictor2],label=train_data['label'])
    
    validation_data = lgb.Dataset(val_data[predictor2],label=val_data['label'])
    #break
#     val_data.reset_index(inplace=True,drop=True)
#     val_data.reset_index(inplace=True)
#     index_df = val_data.groupby('m_id')['index'].apply(list)
    
#     label_package = val_data.groupby('m_id',as_index=False)['label'].agg({'label_id_max':'idxmax'})['label_id_max']

    bst = lgb.train(param, train, valid_sets=[validation_data],early_stopping_rounds=200,
                    verbose_eval=100)
    joblib.dump(bst, 'model_link/lgb_gbdt_%d.pkl'%c)
    c+=1
    #break
# 0.0349
# [2499]	valid_0's binary_logloss: 0.0697551
# [2298]	valid_0's binary_logloss: 0.0684312
# [100]	valid_0's binary_logloss: 0.146277
# [1945]	valid_0's binary_logloss: 0.0693056
# [28]	valid_0's binary_logloss: 0.314403
# [87]	valid_0's binary_logloss: 0.0789237
# (1253344, 702) (313336, 702)
# [100]	valid_0's binary_logloss: 0.0672541
# [200]	valid_0's binary_logloss: 0.064743
# [300]	valid_0's binary_logloss: 0.0642422
# [400]	valid_0's binary_logloss: 0.0639828
# [500]	valid_0's binary_logloss: 0.0637877
# [600]	valid_0's binary_logloss: 0.063694
# [700]	valid_0's binary_logloss: 0.0635975
# [800]	valid_0's binary_logloss: 0.0635226
# [900]	valid_0's binary_logloss: 0.063443
# [1000]	valid_0's binary_logloss: 0.0633719
# [1100]	valid_0's binary_logloss: 0.0633494
# [1200]	valid_0's binary_logloss: 0.0633296
# [1300]	valid_0's binary_logloss: 0.0632968
# [1400]	valid_0's binary_logloss: 0.0632536
# [1500]	valid_0's binary_logloss: 0.0632442
# [1600]	valid_0's binary_logloss: 0.0632364
# [1700]	valid_0's binary_logloss: 0.0632445
# [1800]	valid_0's binary_logloss: 0.0632452
# Early stopping, best iteration is:
# [1624]	valid_0's binary_logloss: 0.0632279

# [100]	valid_0's binary_logloss: 0.0668833
# [200]	valid_0's binary_logloss: 0.0642146
# [300]	valid_0's binary_logloss: 0.0637025
# [400]	valid_0's binary_logloss: 0.0633815
# [500]	valid_0's binary_logloss: 0.0632067
# [600]	valid_0's binary_logloss: 0.0630446
# [700]	valid_0's binary_logloss: 0.0629175
# [800]	valid_0's binary_logloss: 0.062835
# [900]	valid_0's binary_logloss: 0.0627646
# [1000]	valid_0's binary_logloss: 0.0627174
# [11100]	valid_0's binary_logloss: 0.0626483
# [1200]	valid_0's binary_logloss: 0.062619
# [1300]	valid_0's binary_logloss: 0.0625717
# [1400]	valid_0's binary_logloss: 0.0625455

(1253344, 782) (313336, 782)


/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_logloss: 0.0656267
[200]	valid_0's binary_logloss: 0.0625694
[300]	valid_0's binary_logloss: 0.0619294
[400]	valid_0's binary_logloss: 0.0615979
[500]	valid_0's binary_logloss: 0.0613171
[600]	valid_0's binary_logloss: 0.0611661
[700]	valid_0's binary_logloss: 0.0609882
[800]	valid_0's binary_logloss: 0.0608508
[900]	valid_0's binary_logloss: 0.060741
[1000]	valid_0's binary_logloss: 0.0606541
[1100]	valid_0's binary_logloss: 0.0606077
[1200]	valid_0's binary_logloss: 0.0605713
[1300]	valid_0's binary_logloss: 0.0605059
[1400]	valid_0's binary_logloss: 0.0604634
[1500]	valid_0's binary_logloss: 0.0604281
[1600]	valid_0's binary_logloss: 0.0604064
[1700]	valid_0's binary_logloss: 0.0603616
[1800]	valid_0's binary_logloss: 0.0603506
[1900]	valid_0's binary_logloss: 0.0603358
[2000]	valid_0's binary_logloss: 0.0603382
[2100]	valid_0's binary_logloss: 0.0603249
[2200]	valid_0's binary_logloss: 0.0603105
[

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_logloss: 0.0673982
[200]	valid_0's binary_logloss: 0.0645594
[300]	valid_0's binary_logloss: 0.0639123
[400]	valid_0's binary_logloss: 0.0635984
[500]	valid_0's binary_logloss: 0.0633563
[600]	valid_0's binary_logloss: 0.0631864
[700]	valid_0's binary_logloss: 0.0630681
[800]	valid_0's binary_logloss: 0.0629306
[900]	valid_0's binary_logloss: 0.0628349
[1000]	valid_0's binary_logloss: 0.0628099
[1100]	valid_0's binary_logloss: 0.062728
[1200]	valid_0's binary_logloss: 0.0626644
[1300]	valid_0's binary_logloss: 0.0626202
[1400]	valid_0's binary_logloss: 0.0625956
[1500]	valid_0's binary_logloss: 0.0625443
[1600]	valid_0's binary_logloss: 0.0625223
[1700]	valid_0's binary_logloss: 0.0624934
[1800]	valid_0's binary_logloss: 0.0624695
[1900]	valid_0's binary_logloss: 0.0624727
[2000]	valid_0's binary_logloss: 0.0624667
Early stopping, best iteration is:
[1846]	valid_0's binary_logloss: 0.0624532
(1253344,

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


KeyboardInterrupt: 

In [ ]:
data['m_id'].max()

In [ ]:
bst.current_iteration()

In [21]:
# 
# joblib.dump(bst, 'model_link/lgb_dart.pkl')
#import copy
bst = joblib.load('model_link/lgb_gbdt_0.pkl')
#pred2 = bst2.predict(val_data_dart[predictor2])


In [ ]:
def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        span = list(np.linspace(0,1, 100))
        scores = []
        for i in span:
            predict_y =  np.where(y_hat >= i, 1,0)
            score = f1_score(y_true, predict_y)
            scores.append(score)
        #print(scores)
        best_score = max(scores)

In [ ]:
valid_data = val_data[val_data.pred>0.4]
submit = valid_data.groupby('m_id',as_index=False)['pred'].agg({'label_id_max':'idxmax'})
m = submit.shape[0]
n = sum(val_data.groupby('m_id')['label'].sum())

In [ ]:
m

In [ ]:
# 标注错误总结
1 非nil标注为nil
2 kb_id链接到其他词汇
3 可标可不标
4 纯粹标注错误
label=0 预测偏大 纠正
label=1 预测偏小 纠正

In [ ]:
val_data.to_pickle('data/old.pkl')

In [ ]:
val_data = pd.read_pickle('data/t_val.pkl')

In [17]:
pred = bst.predict(val_data[predictor2])
val_data['pred'] = pred

In [ ]:
data[data.label==1]['m_id'].nunique()

In [ ]:
data['m_id'].max()

In [18]:
span = list(np.linspace(0.3,0.5, 10))
scores = []
for i in span:
#     predict_y =  np.where(pred >= i, 1,0)
#     score = f1_score(val_data['label'], predict_y)
#     #print(i,score)
#     scores.append(score)
#     best_score = max(scores)
    n = i
    #val_data.loc[(val_data.label_mean>0.99)&(val_data.label_count>10),'pred']=1
    valid_data = val_data[(val_data.pred>n)]
    v_index = valid_data.loc[valid_data.pred>0.99,'pred'].index

    #valid_data.loc[(valid_data.label_mean==0)&(valid_data.label_count>15),'pred']=0

    submit = valid_data.groupby('m_id',as_index=False)['pred'].agg({'label_id_max':'idxmax'})
    submit_index = np.array(list(set(list(submit['label_id_max'])+list(v_index))))
    m = submit.shape[0]
    n = sum(val_data.groupby('m_id')['label'].sum())
    #hit = valid_data.loc[submit['label_id_max']|valid_data.pred>0.9,:]['label'].sum()
    hit = valid_data.loc[submit_index,:]['label'].sum()
    print(m,n,hit)
    acc = hit/m*0.85
    recall = hit/n*0.85
    print(i,acc,recall,2*acc*recall/(acc+recall))
# 0.358974358974359 0.9064828744358535 0.9083260297984225 0.9074035161448484
# 0.41025641025641024 0.9134305067915026 0.9029272567922875 0.9081485138041677
# 0.4358974358974359 0.9561564043528741 0.9456976585764612 0.9508982739050452
# 0.1794871794871795 0.8884175317656177 0.8817278302937068 0.8850600402300017
# 0.4358974358974359 0.9231731834406925 0.9112650102261912 0.9171804461474181
# 0.358974358974359 0.9152727272727272 0.9174412247129579 0.9163556930918358


50459 49740 46224
0.3 0.7786599020987336 0.7899155609167672 0.784247347777922
50314 49740 46165
0.3222222222222222 0.7799071828914418 0.7889073180538801 0.7843814340256263
50157 49740 46112
0.34444444444444444 0.7814502462268477 0.7880016083634901 0.7847122536212298
50002 49740 46059
0.36666666666666664 0.7829716811327547 0.7870958986731 0.7850283732028633
49840 49740 45988
0.3888888888888889 0.7843057784911718 0.785882589465219 0.7850933922474392
49648 49740 45905
0.4111111111111111 0.7859178617466968 0.7844642139123441 0.7851903650340082
49462 49740 45817
0.43333333333333335 0.7873610044074238 0.7829603940490552 0.7851545331747344
49242 49740 45712
0.45555555555555555 0.7890662442630274 0.7811660635303579 0.7850962801317412
49016 49740 45594
0.47777777777777775 0.7906581524400196 0.7791495778045838 0.7848616792903722
48769 49740 45480
0.5 0.7926756751214911 0.7772014475271412 0.7848622968459734


In [ ]:
valid_data = val_data[val_data.pred>0.4]

submit = valid_data.groupby('m_id',as_index=False)['pred'].agg({'label_id_max':'idxmax'})
val_data['positive'] = 0
val_data.loc[submit['label_id_max'],'positive'] = 1

In [ ]:
a = val_data[val_data.positive!=val_data.label][['text_id','kb_id','train_mention','num_candidates','label','label_mean','label_count','m_label_mean','m_label_count',
          'cos_distance','pred']]
a[a.pred>0.8]

In [ ]:
a[(a.label_mean>=0.99)&(a.label_count>10)]

In [ ]:
val_data[(val_data.m_label_count>10)&(val_data.m_label_mean==1)&(val_data.positive!=val_data.label)]\
[['text_id','kb_id','train_mention','num_candidates','label','label_mean','label_count','m_label_mean','m_label_count',
          'pred']]

In [ ]:
val_data[(val_data.m_label_count>10)&(val_data.m_label_mean==1)&(val_data.positive!=val_data.label)]\
[['text_id','kb_id','train_mention','num_candidates','label','label_mean','label_count','m_label_mean','m_label_count',
          'pred']]

In [ ]:
val_data[val_data.positive!=val_data.label].shape

In [ ]:
val_data.shape

In [ ]:
# a = val_data[val_data.positive!=val_data.label][['text_id','kb_id','train_mention','num_candidates','label','label_mean','label_count','m_label_mean','m_label_count',
#           'pred']]
# a[a.pred>0.8]

In [ ]:
# val_data[['text_id','kb_id','train_mention','label','label_mean','label_count','m_label_mean','m_label_count',
#           'cos_distance','rank_cos_distance','pred']]

In [ ]:
# span = list(np.linspace(0,1, 100))
# scores = []
# for i in span:
#     predict_y =  np.where(pred >= i, 1,0)
#     score = f1_score(val_data['label'], predict_y)
#     #print(i,score)
#     scores.append(score)
# best_score = max(scores)
# n = scores.index(best_score)/100
# print(n)

In [15]:
############################## for submit  #####################################

train = lgb.Dataset(data[predictor2],label=data['label'])
param = {
    #'num_leaves':100,
    'objective':'binary',
    'metric':'binary_logloss',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    #'xgboost_dart_mode':True,
    'colsample_bytree':0.5,
    'n_estimators':2000,
    'min_child_weight':5,
    'subsample_freq':2,
    'num_leaves':32,
    'n_jobs':-1,
}


bst = lgb.train(param, train)
from sklearn.externals import joblib
joblib.dump(bst, 'model_link/lgb_gbdt_4.pkl')

# save model
# load model
#gbm_pickle = joblib.load('lgb.pkl')
# break

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


['model_link/lgb_gbdt_4.pkl']

In [ ]:
predictor2

In [ ]:
# val_data

In [ ]:
list(best_features.sort_values(by=['importance'])['feature'])

In [ ]:
data[['text_id','kb_id','train_mention','label','label_mean','label_count','kbid_rank_label_mean']].sort_values(by='kb_id')

In [ ]:
max(data['m_id'])

In [22]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

ValueError: Length of values does not match length of index

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')